<a href="https://colab.research.google.com/github/fullendmaestro/Llama-Chatbot/blob/main/Llama_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-2 Enhanced Chatbot with Sentiment Analysis and Semantic Search

## Overview

This project implements a conversational AI chatbot using the **Llama-2 model** for text generation, integrated with **sentiment analysis** to improve user interactions. The chatbot is further enhanced by **semantic search** capabilities through **Pinecone** to retrieve contextually relevant information from multiple datasets, allowing it to provide more accurate and personalized responses.

### Key Features:
1. **Llama-2 Text Generation**:
   - The chatbot uses the pre-trained Llama-2 model from Hugging Face for generating conversational responses. This model produces high-quality natural language outputs.
   
2. **Sentiment Analysis**:
   - A sentiment analysis model from Hugging Face (`distilbert-base-uncased-finetuned-sst-2-english`) detects whether user input is positive, negative, or neutral. This enables the chatbot to adjust its tone accordingly, enhancing the interaction quality.

3. **Semantic Search using Pinecone**:
   - Pinecone's vector database is employed to perform semantic searches across three datasets:
     - **QA Dataset**: Frequently asked questions and answers.
     - **Product Dataset**: Product descriptions and metadata.
     - **Troubleshooting Dataset**: Steps and solutions to resolve common issues.
   - The chatbot uses **sentence-transformers** to generate vector embeddings of user queries and document texts, enabling quick and accurate searches based on semantic similarity.

4. **Contextual Conversations**:
   - The chatbot combines historical interactions with search results to generate more context-aware responses, making the conversation feel natural and connected to previous queries.

### Datasets Used:
- **`qa_dataset.csv`**: Contains Q&A pairs, helping the chatbot answer common user questions.
- **`products.csv`**: A dataset of product descriptions used to retrieve product-related information.
- **`troubleshooting.csv`**: Contains troubleshooting steps for resolving technical issues.

### Integration:
- **Hugging Face Transformers**: For text generation (Llama-2) and sentiment analysis models.
- **Pinecone**: For fast, scalable vector-based searches using sentence embeddings.
- **Gradio**: A web-based user interface that allows users to interact with the chatbot in real-time.

### Architecture:
1. **User Input**: The user asks a question or provides feedback.
2. **Sentiment Analysis**: The chatbot analyzes the emotional tone of the input (positive, neutral, or negative).
3. **Semantic Search**: The system queries the Pinecone index for relevant documents across multiple datasets.
4. **Contextual Response Generation**: Llama-2 generates a response based on the search results, chat history, and detected sentiment.
5. **User Response**: The chatbot tailors its reply to reflect the detected sentiment and retrieved information.

---

This project showcases a fully interactive and intelligent chatbot, capable of handling a variety of queries and responding dynamically based on the user's sentiment and contextual search results.



In [ ]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6lg43jn6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6lg43jn6
  Resolved https://github.com/huggingface/transformers to commit e1b150862e66e16acf951edfa13206ffcd1032be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9920673 sha256=591742588da786627741a84c105b3589b3057c7a3123ff537f585197934c0668
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8b0krgr/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [ ]:
!pip -q install openai pinecone-client gradio sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.1/378.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# API Keys from colab secret for Hugging Face, OpenAI, and Pinecone
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
huggingface_api_key = userdata.get('HUGGINGFACEHUB_API_TOKEN')
openai_api_key = userdata.get('OPENAI_API_KEY')
pinecone_region = userdata.get('PINECONE_ENV')

In [ ]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch

In [ ]:
# Hugging Face Authentication
login(token=huggingface_api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Define the paths for the CSV files
products_csv = 'products.csv'
qa_csv = 'qa_dataset.csv'
troubleshooting_csv = 'troubleshooting.csv'



# Load the existing CSV file into a DataFrame
qa_df = pd.read_csv(qa_csv)
products_df = pd.read_csv(products_csv)
troubleshooting_df = pd.read_csv(troubleshooting_csv)

In [ ]:
# Verify the CSV content
print("QA Dataset:")
print(qa_df.head())

print("\nProducts Dataset:")
print(products_df.head())

print("\nTroubleshooting Dataset:")
print(troubleshooting_df.head())

QA Dataset:
                                            Question  \
0  My SmartHome Hub won't connect to Wi-Fi. What ...   
1  The temperature readings on my Smart Thermosta...   
2  My Smart Lights won't turn on or off using the...   
3  The Smart Lock isn't responding to app command...   
4  My Smart Security Camera isn't showing a live ...   

                                              Answer  
0  I understand you're having trouble connecting ...  
1  I'm sorry to hear your Smart Thermostat is sho...  
2  I see you're having issues controlling your Sm...  
3  I understand your Smart Lock isn't responding ...  
4  I'm sorry to hear you're not getting a live fe...  

Products Dataset:
                           Title  \
0              SmartHome Hub Pro   
1      EcoTherm Smart Thermostat   
2      LuminaGlow Smart Bulb Set   
3         SecureGuard Smart Lock   
4  ClearView Pro Security Camera   

                                         Description  
0  Control your entire smart h

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import openai
from sentence_transformers import SentenceTransformer


# Load the sentence transformer model
embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Pinecone index name
index_name = "llama-quest-index"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# # Check if the index exists, if not, create it
# if index_name not in pc.list_indexes():
#     pc.create_index(
#         name=index_name,
#         dimension=768,  # Embedding dimension for 'sentence-transformers/multi-qa-mpnet-base-cos-v1'
#         metric="cosine",
#         spec=ServerlessSpec(
#             cloud="aws",
#             region=pinecone_region
#         )
#     )

In [ ]:
# Connect to the Pinecone index
index = pc.Index(index_name)

In [ ]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,
)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:

# Load a sentiment analysis model from Hugging Face
sentiment_analyzer = pipeline('sentiment-analysis')

# New function to analyze the sentiment of the user's input
def analyze_sentiment(user_input):
    result = sentiment_analyzer(user_input)[0]
    sentiment = result['label']
    return sentiment.lower()  # Convert sentiment to lowercase to match categories

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Debugging function to print the current state of history
def print_history_debug(history):
    print("\n----- Current History -----")
    for idx, (user_input, bot_response) in enumerate(history):
        print(f"Turn {idx+1}:")
        print(f"User: {user_input}")
        print(f"Bot: {bot_response}")
    print("----------------------------\n")

In [ ]:
# Embed and store documents in Pinecone
def embed_and_store_documents(df, text_column, namespace, metadata_columns=None):
    if metadata_columns is None:
        metadata_columns = []  # Ensure it's an empty list if not provided

    for idx, row in df.iterrows():
        # Handle text_column being a list or string
        text = row[text_column] if isinstance(text_column, str) else ' '.join([str(row[col]) for col in text_column])

        # Embed the text using the sentence-transformers model
        embedding = embedding_model.encode(text).tolist()  # Convert to list format for Pinecone

        # Extract metadata
        metadata = {col: row[col] for col in metadata_columns}  # Handle metadata columns

        # Insert into Pinecone with unique ID (index)
        index.upsert([(str(idx), embedding, metadata)], namespace=namespace)


In [ ]:
# Function to load datasets and index them correctly
def index_datasets():
    # Load datasets
    qa_df = pd.read_csv('qa_dataset.csv')
    products_df = pd.read_csv('products.csv')
    troubleshooting_df = pd.read_csv('troubleshooting.csv')

    # Index QA dataset: Use "Answer" as document, "Question" as metadata
    embed_and_store_documents(qa_df, 'Answer', 'qa', metadata_columns=['Question'])

    # Index Products dataset: Use "Description" as document, "Title" as metadata
    embed_and_store_documents(products_df, 'Description', 'products', metadata_columns=['Title'])

    # Index Troubleshooting dataset: Concatenate "Issue" and "Steps" for document, "Device" as metadata
    embed_and_store_documents(troubleshooting_df, ['Issue', 'Steps'], 'troubleshooting', metadata_columns=['Device'])

# Run the indexing process
# index_datasets()


In [ ]:
# Semantic search function using Pinecone
def semantic_search(query, namespace):
    # Use sentence-transformers to encode the query
    query_embedding = embedding_model.encode(query).tolist()

    # Query Pinecone
    results = index.query(
        vector=query_embedding,  # The query embedding
        top_k=3,  # Top 3 results
        include_metadata=True,  # Include metadata in the results
        namespace=namespace  # Search in the specified namespace
    )

    return results

In [ ]:
# Modify the generate_with_context function to take sentiment into account
def generate_with_context(question, search_results, history, sentiment):
    # Prepare the context from search results and history
    if search_results:
        best_match = search_results[0]['metadata']  # Assuming metadata holds relevant information
        context = f"Context: {best_match}\n\n"
    else:
        context = ""

    # Format the history into the prompt
    conversation_history = ""
    for user_input, bot_response in history:
        conversation_history += f"User: {user_input}\nAI: {bot_response}\n"

    # Combine context, history, and new question
    full_prompt = f"{context}{conversation_history}User: {question}\nAI:"

    # Adjust the response based on the sentiment
    if sentiment == 'negative':
        full_prompt = f"Be empathetic in your responses.\n{full_prompt}"
    elif sentiment == 'positive':
        full_prompt = f"Be friendly and upbeat in your responses.\n{full_prompt}"
    else:
        full_prompt = f"Be neutral and formal in your responses.\n{full_prompt}"

    # Generate the response using Llama-2
    response = llama_pipeline(
        full_prompt,
        max_new_tokens=150,  # Specify how many tokens the model should generate in the response
        do_sample=True
    )[0]['generated_text']

    # Remove the original prompt from the generated response
    response_text = response[len(full_prompt):].strip()  # Only return the model's response text
    return response_text

In [ ]:
# Main function to handle a user's question and provide an answer
def answer_question(question, history):
    # Perform semantic search across multiple namespaces (QA, products, troubleshooting)
    search_results = []
    try:
        search_results.extend(semantic_search(question, 'qa').matches)
        search_results.extend(semantic_search(question, 'products').matches)
        search_results.extend(semantic_search(question, 'troubleshooting').matches)
    except Exception as e:
        print(f"Error during semantic search: {e}")
        return "Error during semantic search."

    # Analyze sentiment of the user's question
    sentiment = analyze_sentiment(question)

    # Generate answer using the search results as context, conversation history, and sentiment
    try:
        response = generate_with_context(question, search_results, history, sentiment)
    except Exception as e:
        print(f"Error during response generation: {e}")
        return f"Error during response generation: {e}"

    return response

In [ ]:
import gradio as gr

# Gradio chat interface
def gradio_chat_interface(question, history):

    response = answer_question(question, history)
    return response

In [ ]:
# Create a Gradio Interface
interface = gr.ChatInterface(fn=gradio_chat_interface)

In [ ]:
# Launch the Gradio Interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fdaa8d6426ae0a2f1e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# res = answer_question("What is the price of the iPhone 12?", [])
# res